In [9]:
import pickle
from typing import Tuple

import numpy as np
import pandas as pd
import shap
import tensorflow as tf
from keras.layers import LSTM, Dense
from keras.models import Sequential, load_model
from keras.preprocessing.sequence import TimeseriesGenerator


import sys

sys.path.append("../src/")
from models.training_helpers import get_timeseries_generator, get_features_and_target

# tf.compat.v1.disable_v2_behavior()
print("SHAP version is:", shap.__version__)
print("Tensorflow version is:", tf.__version__)

SHAP version is: 0.44.0
Tensorflow version is: 2.15.0


In [10]:
TRAINING_SET = "../data/trusted/training_set.csv"
VALIDATION_SET = "../data/trusted/validation_set.csv"
TEST_SET = "../data/trusted/test_set.csv"

MODEL_PATH = "../models_save/lstm"

In [6]:
X_train, y_train = get_features_and_target(pd.read_csv(TRAINING_SET, index_col=0))
X_val, y_val = get_features_and_target(pd.read_csv(VALIDATION_SET, index_col=0))

In [7]:
X_train

,onpromotion,dcoilwtico,cluster,Holiday,Normal,Additional,Event,AUTOMOTIVE,BABY CARE,BEAUTY,...,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD,D,C,B,E,A
0,0.000000,0.792965,0.7500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000000,0.792965,0.7500,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.000000,0.792965,0.7500,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.000000,0.792965,0.7500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.000000,0.792965,0.7500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1849443,0.001350,0.293261,0.8125,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1849444,0.010796,0.293261,0.8125,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1849445,0.000000,0.293261,0.8125,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1849446,0.000000,0.293261,0.8125,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [8]:
y_train

,sales
0,0.000
1,0.000
2,0.000
3,0.000
4,0.000
...,...
1849443,520.536
1849444,474.000
1849445,8.000
1849446,29.000


In [37]:
history_input = 30
n_features = len(X_train.columns)
batch_size = 1

generator = TimeseriesGenerator(
    X_train[1500000:].to_numpy(),
    y_train[1500000:].to_numpy(),
    length=history_input,
    batch_size=batch_size,
)

In [38]:
len(generator)

349418

In [39]:
def lstm(load_model=False, model_name="lstm"):
    if load_model:
        pickled_model = pickle.load(open(f"{MODEL_PATH}/{model_name}.h5", "rb"))
    else:
        model = keras.Sequential()
        model.add(LSTM(50, activation="relu", input_shape=(history_input, n_features)))
        model.add(Dense(1, activation="linear"))
        model.compile(
            optimizer="adam",
            loss="mean_squared_logarithmic_error",
            metrics=["accuracy"],
        )

        model.fit(generator, epochs=1, batch_size=512)

        pickle.dump(model, open(f"{MODEL_PATH}/{model_name}.h5", "wb"))
        pickled_model = model

    return pickled_model

# Test LSTM

In [40]:
from sklearn.metrics import mean_squared_log_error

In [41]:
m = lstm(True)

349418/349418 [==============================] - 3755s 11ms/step - loss: 16.3523 - accuracy: 0.1815


ValueError: in user code:

    File "/home/acaille/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/home/acaille/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/acaille/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/home/acaille/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/home/acaille/miniconda3/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/acaille/miniconda3/lib/python3.11/site-packages/keras/src/engine/input_spec.p

In [ ]:
generator_val = TimeseriesGenerator(
    X_val.to_numpy(), y_val.to_numpy(), length=history_input, batch_size=batch_size
)

In [ ]:
pred = m.predict(X_val_lstm)

In [ ]:
pred = pred.clip(0)

In [ ]:
# Model evaluation

reshaped_y_val = y_val_lstm.reshape((y_val_lstm.shape[0], y_val_lstm.shape[1]))

lstm_f1 = round(mean_squared_log_error(reshaped_y_val, pred), 2)
print("mean log squared error : ", lstm_f1)

In [ ]:
m.summary()

# Explicability

In [13]:
shap_exp = shap.DeepExplainer(m, X_train_lstm[:500])

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


In [14]:
shap_values = shap_exp.shap_values(X_val_lstm[:50])
shap.initjs()  # initialize javascript to enable plots

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


StagingError: in user code:

    File "/home/acaille/miniconda3/lib/python3.11/site-packages/shap/explainers/_deep/deep_tf.py", line 252, in grad_graph  *
        x_grad = tape.gradient(out, shap_rAnD)

    LookupError: gradient registry has no entry for: shap_TensorListStack


In [ ]:
shap.summary_plot(shap_values, X_val)